# <center>Machine Learning Project Code</center>

<a class="anchor" id="top"></a>

## <center>*03 - K-Fold*</center>

** **



# Table of Contents  <br>


1. [Importing Libraries & Data](#1.-Importing-Libraries-&-Data) <br><br>
    
2. [Cross Validation](#2.-Cross-Validation) <br><br>

3. [Final Predictions](#3.-Final-Predictions) <br><br>

** **

This notebook will consist of the implementation of Stratified K-Fold. It will use the same techniques to fill missing values and treat outliers as Notebook 02. Feature Selection will only be performed in said notebook, and the selected features there will be used here, due to computational complexity and time constraints.

Data Scientist Manager: António Oliveira, **20211595**

Data Scientist Senior: Tomás Ribeiro, **20240526**

Data Scientist Junior: Gonçalo Pacheco, **20240695**

Data Analyst Senior: Gonçalo Custódio, **20211643**

Data Analyst Junior: Ana Caleiro, **20240696**


** ** 

# 1. Importing Libraries & Data
In this section, we set up the foundation for our project by importing the necessary Python libraries and loading the dataset. These libraries provide the tools for data manipulation, visualization, and machine learning modeling throughout the notebook. Additionally, we import the historical claims dataset, which forms the core of our analysis. 

In [1]:
import pandas as pd
import numpy as np

# Train-Test Split
from sklearn.model_selection import StratifiedKFold

# Scaler
from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler,
    RobustScaler
)

import utils2 as p 

# Models
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, \
    GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier 
from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier


# Metrics
from sklearn.metrics import classification_report
import metrics as m
from sklearn.metrics import f1_score, precision_score, recall_score

pd.set_option('display.max_columns', None)
import time

# Suppress Warnings
import warnings
warnings.filterwarnings("ignore")

**Import Data**

In [2]:
# Load training data
df = pd.read_csv('./data/train_data_EDA.csv', index_col = 'Claim Identifier')

# Load testing data
test1 = pd.read_csv('./data/test_data_EDA.csv', index_col = 'Claim Identifier')

# Display the first 3 rows of the training data
df.head(3)

,Age at Injury,Alternative Dispute Resolution,Attorney/Representative,Average Weekly Wage,Birth Year,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,Gender Enc,Accident Date Year,Accident Date Month,Accident Date Day,Accident Date Day of Week,Assembly Date Year,Assembly Date Month,Assembly Date Day,Assembly Date Day of Week,C-2 Date Year,C-2 Date Month,C-2 Date Day,C-2 Date Day of Week,Accident to Assembly Time,Assembly to C-2 Time,Accident to C-2 Time,WCIO Codes,Insurance,Zip Code Valid,Industry Sector,Age Group
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5393875,31.0,N,N,0.00,1988.0,NaN,NEW HAMPSHIRE INSURANCE CO,1A. PRIVATE,1,ST. LAWRENCE,N,SYRACUSE,NaN,M,NaN,44.0,I,27,10,62,1.0,0,2019.0,12.0,30.0,0.0,2020,1,1,2,2019.0,12.0,31.0,1.0,2.0,1.0,1.0,271062,1,0,Retail and Wholesale,1
5393091,46.0,N,Y,1745.93,1973.0,2020-01-14,ZURICH AMERICAN INSURANCE CO,1A. PRIVATE,3,WYOMING,N,ROCHESTER,2020-02-21,F,4.0,23.0,I,97,49,38,4.0,1,2019.0,8.0,30.0,4.0,2020,1,1,2,2020.0,1.0,1.0,2.0,124.0,0.0,124.0,974938,1,0,Manufacturing and Construction,1
5393889,40.0,N,N,1434.80,1979.0,NaN,INDEMNITY INSURANCE CO OF,1A. PRIVATE,3,ORANGE,N,ALBANY,NaN,M,NaN,56.0,II,79,7,10,6.0,0,2019.0,12.0,6.0,4.0,2020,1,1,2,2020.0,1.0,1.0,2.0,26.0,0.0,26.0,79710,1,0,Business Services,1


# 2. Cross Validation

<a href="#top">Top &#129033;</a>

In [3]:
# Split the DataFrame into features (X) and target variable (y)
X = df.drop('Claim Injury Type', axis=1) 
y = df['Claim Injury Type']  

**Stratified K-Fold**

In [4]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Initialize model
model = XGBClassifier(n_estimators = 150, max_depth = 8,
                     learning_rate = 0.3, colsample_bytree = 0.8)


for train_index, val_index in skf.split(X, y):

    start_time = time.time()
    
    X_train, X_val = X.iloc[train_index].copy(), X.iloc[val_index].copy()
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    test = test1.copy()
    
    # ENCODING
    X_train['Alternative Dispute Resolution Bin'] = X_train['Alternative Dispute Resolution'].replace({'N': 0, 'Y': 1, 'U': 1})
    X_val['Alternative Dispute Resolution Bin'] = X_val['Alternative Dispute Resolution'].replace({'N': 0, 'Y': 1, 'U': 1})
    test['Alternative Dispute Resolution Bin'] = test['Alternative Dispute Resolution'].replace({'N': 0, 'Y': 1, 'U': 1})
    
    X_train['Attorney/Representative Bin'] = X_train['Attorney/Representative'].replace({'N': 0, 'Y': 1})
    X_val['Attorney/Representative Bin'] = X_val['Attorney/Representative'].replace({'N': 0, 'Y': 1})
    test['Attorney/Representative Bin'] = test['Attorney/Representative'].replace({'N': 0, 'Y': 1})
    
    train_carriers = set(X_train['Carrier Name'].unique())
    test_carriers = set(test['Carrier Name'].unique())
    common_categories = train_carriers.intersection(test_carriers)
    common_category_map = {category: idx + 1 for idx, 
                       category in enumerate(common_categories)}
        
    X_train['Carrier Name Enc'] = X_train['Carrier Name'].map(common_category_map).fillna(0).astype(int)
    X_val['Carrier Name Enc'] = X_val['Carrier Name'].map(common_category_map).fillna(0).astype(int)
    test['Carrier Name Enc'] = test['Carrier Name'].map(common_category_map).fillna(0).astype(int)
    
    X_train, X_val, test = p.encode(X_train, X_val, test, 'Carrier Name Enc', 'count')
    
    X_train, X_val, test = p.encode(X_train, X_val, test, 'Carrier Type', 'count')
    X_train, X_val, test = p.encode(X_train, X_val, test, 'Carrier Type', 'OHE')
    
    X_train, X_val, test = p.encode(X_train, X_val, test, 'County of Injury', 'count')
    
    X_train['COVID-19 Indicator Enc'] = X_train['COVID-19 Indicator'].replace({'N': 0, 'Y': 1})
    X_val['COVID-19 Indicator Enc'] = X_val['COVID-19 Indicator'].replace({'N': 0, 'Y': 1})
    test['COVID-19 Indicator Enc'] = test['COVID-19 Indicator'].replace({'N': 0, 'Y': 1})
    
    X_train, X_val, test = p.encode(X_train, X_val, test, 'District Name', 'count')
    
    X_train, X_val, test = p.encode(X_train, X_val, test, 'Gender', 'OHE')
    
    X_train, X_val, test = p.encode(X_train, X_val, test, 'Medical Fee Region', 'count')
    
    X_train, X_val, test = p.encode(X_train, X_val, test, 'Industry Sector', 'count')
    
    drop = ['Alternative Dispute Resolution', 'Attorney/Representative', 'Carrier Type', 'County of Injury',
            'COVID-19 Indicator', 'District Name', 'Gender', 'Carrier Name',
            'Medical Fee Region', 'Industry Sector']
    
    X_train.drop(columns = drop, axis = 1, inplace = True)
    X_val.drop(columns = drop, axis = 1, inplace = True)
    test.drop(columns = drop, axis = 1, inplace = True)
    
    # MISSING VALUES
    X_train['C-3 Date Binary'] = X_train['C-3 Date'].notna().astype(int)
    X_val['C-3 Date Binary'] = X_val['C-3 Date'].notna().astype(int)
    test['C-3 Date Binary'] = test['C-3 Date'].notna().astype(int)
    
    X_train['First Hearing Date Binary'] = X_train['First Hearing Date'].notna().astype(int)
    X_val['First Hearing Date Binary'] = X_val['First Hearing Date'].notna().astype(int)
    test['First Hearing Date Binary'] = test['First Hearing Date'].notna().astype(int)
    
    drop = ['C-3 Date', 'First Hearing Date']
    X_train.drop(columns = drop, axis = 1, inplace = True)
    X_val.drop(columns = drop, axis = 1, inplace = True)
    test.drop(columns = drop, axis = 1, inplace = True)
    
    X_train['IME-4 Count'] = X_train['IME-4 Count'].fillna(0)
    X_val['IME-4 Count'] = X_val['IME-4 Count'].fillna(0)
    test['IME-4 Count'] = test['IME-4 Count'].fillna(0)
    
    X_train['Industry Code'] = X_train['Industry Code'].fillna(0)
    X_val['Industry Code'] = X_val['Industry Code'].fillna(0)
    test['Industry Code'] = test['Industry Code'].fillna(0)
    
    p.fill_dates(X_train, [X_val, test], 'Accident Date')
    p.fill_dates(X_train, [X_val, test], 'C-2 Date')
    
    p.fill_dow([X_train, X_val, test], 'Accident Date')
    p.fill_dow([X_train, X_val, test], 'C-2 Date')
    
    X_train = p.fill_missing_times(X_train, ['Accident to Assembly Time', 
                             'Assembly to C-2 Time',
                             'Accident to C-2 Time'])

    X_val = p.fill_missing_times(X_val, ['Accident to Assembly Time', 
                             'Assembly to C-2 Time',
                             'Accident to C-2 Time'])

    test = p.fill_missing_times(test, ['Accident to Assembly Time', 
                             'Assembly to C-2 Time',
                             'Accident to C-2 Time'])
    
    p.fill_birth_year([X_train, X_val, test])
    
    
    # variables
    num = ['Age at Injury', 'Average Weekly Wage', 'Birth Year',
       'IME-4 Count', 'Number of Dependents', 'Accident Date Year',
       'Accident Date Month', 'Accident Date Day', 
       'Assembly Date Year', 'Assembly Date Month', 
       'Assembly Date Day', 'C-2 Date Year', 'C-2 Date Month',
       'C-2 Date Day', 'Accident to Assembly Time',
       'Assembly to C-2 Time', 'Accident to C-2 Time']
      # 'Wage to Age Ratio', 'Average Weekly Wage Sqrt',
      # 'IME-4 Count Log', 'IME-4 Count Double Log']

 
    categ = [var for var in X_train.columns if var not in num]

    categ_count_encoding = ['Carrier Name Enc', 'Carrier Type Enc',
                            'County of Injury Enc', 'District Name Enc',
                            'Medical Fee Region Enc', 
                            'Industry Sector Enc']


    categ_label_bin = [var for var in X_train.columns if var
                       in categ and var not in categ_count_encoding]

    num_count_enc = num + categ_count_encoding


    # scale
    robust = RobustScaler()
    
    X_train_num_count_enc_RS = robust.fit_transform(X_train[num_count_enc])
    X_train_num_count_enc_RS = pd.DataFrame(X_train_num_count_enc_RS, columns=num_count_enc, index=X_train.index)
    X_val_num_count_enc_RS = robust.transform(X_val[num_count_enc])
    X_val_num_count_enc_RS = pd.DataFrame(X_val_num_count_enc_RS, columns=num_count_enc, index=X_val.index)
    test_num_count_enc_RS = robust.transform(test[num_count_enc])
    test_num_count_enc_RS = pd.DataFrame(test_num_count_enc_RS, columns=num_count_enc, index=test.index)

    X_train_RS = pd.concat([X_train_num_count_enc_RS, 
                            X_train[categ_label_bin]], axis=1)
    X_val_RS = pd.concat([X_val_num_count_enc_RS, 
                          X_val[categ_label_bin]], axis=1)
    test_RS = pd.concat([test_num_count_enc_RS, 
                         test[categ_label_bin]], axis=1)
    
    p.ball_tree_impute([X_train_RS, X_val_RS, test_RS], 
                       'Average Weekly Wage')
    

    # OUTLIERS
    
#     X_train['Average Weekly Wage Sqrt'] = np.sqrt(X_train['Average Weekly Wage'])
#     X_val['Average Weekly Wage Sqrt'] = np.sqrt(X_val['Average Weekly Wage'])
#     test['Average Weekly Wage Sqrt'] = np.sqrt(test['Average Weekly Wage'])
    
#     upper_limit = X_train['Average Weekly Wage'].quantile(0.99)
#     lower_limit = X_train['Average Weekly Wage'].quantile(0.01)
#     X_train['Average Weekly Wage'] = X_train['Average Weekly Wage'].clip(lower = lower_limit
#                                                                   , upper=upper_limit)
    
#     lower_limit = X_train['Birth Year'].quantile(0.01)
#     X_train['Birth Year'] = X_train['Birth Year'].clip(lower = lower_limit)
    
#     X_train['IME-4 Count Log'] = np.log1p(X_train['IME-4 Count'])
#     X_train['IME-4 Count Double Log'] = np.log1p(X_train['IME-4 Count Log'])
#     X_val['IME-4 Count Log'] = np.log1p(X_val['IME-4 Count'])
#     X_val['IME-4 Count Double Log'] = np.log1p(X_val['IME-4 Count Log'])
#     test['IME-4 Count Log'] = np.log1p(test['IME-4 Count'])
#     test['IME-4 Count Double Log'] = np.log1p(test['IME-4 Count Log'])
    
#     lower_limit = X_train['Accident Date Year'].quantile(0.01)
#     X_train['Accident Date Year'] = X_train['Accident Date Year'].clip(lower = lower_limit)
    
#     lower_limit = X_train['C-2 Date Year'].quantile(0.01)
#     X_train['C-2 Date Year'] = X_train['C-2 Date Year'].clip(lower = lower_limit)


    # Train the model
    model.fit(X_train_RS, y_train)
    
    # Make predictions
    pred_train = model.predict(X_train_RS)
    pred_val = model.predict(X_val_RS)
    
    # Evaluate Metrics
    m.metrics(y_train, pred_train , y_val, pred_val)

    # Time
    end_time = time.time()
    elapsed_time = round((end_time - start_time) / 60, 2)
    print(f'This Fold took {elapsed_time} minutes')

______________________________________________________________________
                                TRAIN                                 
----------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.92      0.76      0.83     11229
           1       0.88      0.98      0.93    261970
           2       0.84      0.22      0.35     62015
           3       0.79      0.93      0.86    133656
           4       0.82      0.77      0.79     43452
           5       1.00      0.53      0.69      3790
           6       1.00      1.00      1.00        87
           7       1.00      1.00      1.00       423

    accuracy                           0.85    516622
   macro avg       0.91      0.77      0.81    516622
weighted avg       0.85      0.85      0.82    516622

______________________________________________________________________
                                VALIDATION                       

______________________________________________________________________
                                TRAIN                                 
----------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.93      0.75      0.83     11228
           1       0.88      0.98      0.93    261970
           2       0.83      0.22      0.35     62015
           3       0.79      0.93      0.85    133657
           4       0.82      0.77      0.79     43452
           5       1.00      0.52      0.69      3790
           6       1.00      1.00      1.00        88
           7       1.00      1.00      1.00       423

    accuracy                           0.85    516623
   macro avg       0.91      0.77      0.80    516623
weighted avg       0.85      0.85      0.82    516623

______________________________________________________________________
                                VALIDATION                       

In [5]:
import play_song as s
s.play_('audio.mp3')

Input #0, wav, from '/var/folders/mm/fxsq_1490x9dd2w76tqvt3kr0000gn/T/tmpnwfr9u7g.wav':
  Duration: 00:00:10.00, bitrate: 1536 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 48000 Hz, 2 channels, s16, 1536 kb/s


# 3. Final Predictions

<a href="#top">Top &#129033;</a>

In [6]:
test_filtered = test_RS[X_train_RS.columns]

In [7]:
test_filtered['Claim Injury Type'] = model.predict(test_filtered)

Map Predictions to Original Values

In [8]:
label_mapping = {
    0: "1. CANCELLED",
    1: "2. NON-COMP",
    2: "3. MED ONLY",
    3: "4. TEMPORARY",
    4: "5. PPD SCH LOSS",
    5: "6. PPD NSL",
    6: "7. PTD",
    7: "8. DEATH"
}


test_filtered['Claim Injury Type'] = test_filtered['Claim Injury Type'].replace(label_mapping)

In [9]:
# Count unique values in column 'Claim Injury Type'
test_filtered['Claim Injury Type'].value_counts() 

Claim Injury Type
2. NON-COMP        312755
4. TEMPORARY        52097
3. MED ONLY         14286
1. CANCELLED         6399
5. PPD SCH LOSS      2406
8. DEATH               28
6. PPD NSL              4
Name: count, dtype: int64

In [10]:
# Extract the target variable 'Claim Injury Type' from the test dataset for prediction
predictions = test_filtered['Claim Injury Type']

In [11]:
# Assign a descriptive name for easy reference
name = 'XGB_param_all_feat'

# Save the predictions to a CSV file.
predictions.to_csv(f'./pred/{name}.csv')